In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0

dense_layers = [0]
layer_sizes = [64]
conv_layers = [3]

for dense_layer in dense_layers:
    for layer_size in layer_sizes:
        for conv_layer in conv_layers:
            NAME = "{}-conv-{}-nodes-{}-dense-{}".format(conv_layer, layer_size, dense_layer, int(time.time()))
            print(NAME)

            model = Sequential()

            model.add(Conv2D(layer_size, (3, 3), input_shape=X.shape[1:]))
            model.add(Activation('relu'))
            model.add(MaxPooling2D(pool_size=(2, 2)))

            for l in range(conv_layer-1):
                model.add(Conv2D(layer_size, (3, 3)))
                model.add(Activation('relu'))
                model.add(MaxPooling2D(pool_size=(2, 2)))

            model.add(Flatten())

            for _ in range(dense_layer):
                model.add(Dense(layer_size))
                model.add(Activation('relu'))

            model.add(Dense(1))
            model.add(Activation('sigmoid'))

            tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

            model.compile(loss='binary_crossentropy',
                          optimizer='adam',
                          metrics=['accuracy'],
                          )

            model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])

model.save('NSR180-CNN.model')

3-conv-64-nodes-0-dense-1548724546
Train on 639 samples, validate on 275 samples
Epoch 1/10
639/639 [==============================] - 8s 13ms/step - loss: 0.6370 - acc: 0.6729 - val_loss: 0.4980 - val_acc: 0.8836
Epoch 2/10
639/639 [==============================] - 8s 12ms/step - loss: 0.3144 - acc: 0.8905 - val_loss: 0.1681 - val_acc: 0.9564
Epoch 3/10
639/639 [==============================] - 8s 12ms/step - loss: 0.0999 - acc: 0.9703 - val_loss: 0.0596 - val_acc: 0.9782
Epoch 4/10
480/639 [=====================>........] - ETA: 1s - loss: 0.0344 - acc: 0.9917

In [ ]:
import cv2
import tensorflow as tf

In [ ]:
CATEGORIES = ["oksensor", "noksensor"]  # will use this to convert prediction num to string value

In [ ]:
def prepare(filepath):
    IMG_SIZE = 50  # 50 in txt-based
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)  # read in the image, convert to grayscale
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))  # resize image to match model's expected sizing
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)  # return the image with shaping that TF wants.

In [ ]:
model = tf.keras.models.load_model("NSR180-CNN.model")

In [ ]:
#print(prepare('oksensor.jpg'))

In [ ]:
prediction = model.predict([prepare('oksensor.jpg')])  # REMEMBER YOU'RE PASSING A LIST OF THINGS YOU WISH TO PREDICT

In [ ]:
prediction

In [ ]:
prediction[0][0]

In [ ]:
print(CATEGORIES[int(prediction[0][0])])

In [ ]:
prediction = model.predict([prepare('noksensor.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

In [ ]:
prediction = model.predict([prepare('noksensor1.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])

In [ ]:
prediction = model.predict([prepare('oksensor1.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])